# Import eines Pickle-Files

In [1]:
import pandas as pd
import pickle
import os

rootdir = os.getcwd() + "/data/training/"

print(rootdir)

data = []
labels = []

for subdir, dirs, files in os.walk(rootdir):
    for fl in files:
        if fl.endswith('.pkl'):
            # string split for event labelling
            str_arr = fl.split('_')
            label   = str_arr[1]

            # read data and append it to variable
            infile = open((rootdir + fl),'rb')
            new_dict = pickle.load(infile, encoding='latin1')

            data.append(new_dict)
            labels.append(label)


/Users/maximilianrohleder-kirsch/angewandte-ki-cog-village/data/training/


# Read and combine data of some activities

In [20]:
import numpy as np 
from collections import defaultdict

keys = data[0].keys()

unique_keys = []
[unique_keys.append(k) for k in keys if k not in unique_keys]

bending = []
bending_dict = defaultdict(list)

# Collecting Bending data
for i in range(0, 9):
    for key in data[i].keys():
        x = data[i][key]
        y = key
        bending_dict[y].append(x)

concat_data = defaultdict(list)

for k in unique_keys:
    for i in bending_dict[k]:
        concat_data[k].append(i)

    concat_data[k] = np.vstack(concat_data[k])

# ToDo: Mean, Varianz, Minimum, Maximum, Differenz Minimum - Maximum



# Alternative Way of concetanating the data

In [6]:
import numpy as np 
from collections import defaultdict

ACTIVITIES = { 
    "Bending" : 0,
    "Lying": 1,
    "Sitting": 2,
    "Squatting": 3,
    "Standing": 4,
    "Walking": 5,
    "Bring": 6,
    "CleanFloor": 7,
    "CleanSurface": 8,
    "CloseBigBox": 9,
    "CloseDoor": 10,
    "CloseDrawer": 11,
    "CloseLidByRotate": 12,
    "CloseOtherLid": 13,
    "CloseSmallBox": 14,
    "CloseTapWater": 15,
    "Drink": 16,
    "DryOffHand": 17,
    "DryOffHandByShake": 18,
    "EatSmall": 19,
    "Gargle": 20,
    "GettingUp": 21,
    "Hang": 22,
    "LyingDown": 23,
    "OpenBag": 24,
    "OpenBigBox": 25,
    "OpenDoor": 26,
    "OpenDrawer": 27,
    "OpenLidByRotate": 28,
    "OpenOtherLid": 29,
    "OpenSmallBox": 30,
    "OpenTapWater": 31,
    "PlugIn": 32,
    "PressByGrasp": 33,
    "PressFromTop": 34,
    "PressSwitch": 35,
    "PutFromBottle": 36,
    "PutFromTapWater": 37,
    "PutHighPosition": 38,
    "PutOnFloor": 39,
    "Read": 40,
    "Rotate": 41,
    "RubHands": 42,
    "ScoopPut": 43,
    "SittingDown": 44,
    "SquattingDown": 45,
    "StandingUp": 46,
    "StandUpFromSquatting": 47,
    "TakeFromFloor": 48,
    "TakeFromHighPosition": 49,
    "TakeOffJacket": 50,
    "TakeOut": 51,
    "TalkByTelephone": 52,
    "ThrowOut": 53,
    "ThrowOutWater": 54,
    "TouchSmartPhoneScreen": 55,
    "Type": 56,
    "Unhang": 57,
    "Unplug": 58,
    "WearJacket": 59,
    "Write": 60 
}

keys = list(data[0].keys())
unique_labels = list(set(labels))

activity_multi_arr = []
activity_labels = unique_labels

for label in unique_labels:
    data_total = defaultdict(list)

    for dt in data:
        keys = list(dt.keys())
        if dt["label"] == ACTIVITIES[label]:
            for k in keys:
                data_total[k].append(dt[k])

    for k in keys:
        data_total[k] = np.vstack(data_total[k])

    activity_multi_arr.append(data_total)


AttributeError: 'list' object has no attribute 'shape'